In [3]:
!pip install -q crewai crewai-tools chromadb sentence-transformers 'crewai[tools]'
!pip install -q langchain langchain-community langchain-huggingface
!pip install -q accelerate transformers
!pip install -q google-search-results python-dotenv
!pip install -q requests beautifulsoup4
!pip install -q litellm google-generativeai

In [4]:
import os
import warnings
import json
from typing import List, Dict, Optional, Any
from collections import deque
from datetime import datetime

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM

# ✅ CORRECT: Import tools from crewai_tools
from crewai_tools import (
    FileReadTool,
    ScrapeWebsiteTool,
    MDXSearchTool,
    SerperDevTool
)
from crewai.tools import tool

# Vector DB and embeddings
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# LangChain for LLM integration
from langchain_huggingface import HuggingFaceEndpoint
from langchain_text_splitters import RecursiveCharacterTextSplitter # Changed import path

# Web search
import requests
from bs4 import BeautifulSoup

warnings.filterwarnings('ignore')
print("All libraries imported successfully!")

✅ All libraries imported successfully!


In [5]:
# ENVIRONMENT SETUP
from google.colab import userdata

# Get API keys from Colab secrets
# To add secrets: Click icon in left sidebar > Add new secret
try:
    SERPER_API_KEY = userdata.get('SERPER_API_KEY')
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except:
    print("API keys not found in Colab secrets.")
    print("Please add SERPER_API_KEY and GEMINI_API_KEY in Colab secrets.")
    print("\nFor now, enter them manually:")
    SERPER_API_KEY = input("Enter SERPER_API_KEY: ")
    GEMINI_API_KEY = input("Enter GEMINI_API_KEY: ")

# Set environment variables
os.environ['SERPER_API_KEY'] = SERPER_API_KEY
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY  # Set Gemini key

print("Environment configured!")
print(f"   - Serper API: {'✓' if SERPER_API_KEY else '✗'}")
print(f"   - Gemini API: {'✓' if GEMINI_API_KEY else '✗'}")

✅ Environment configured!
   - Serper API: ✓
   - Gemini API: ✓


In [6]:
# CONTEXT ENGINEERING MODULE


class ContextEngineeringModule:
    """
    Advanced context management system with:
    - Short-term memory (rolling window buffer)
    - Long-term memory (vector database)
    - Semantic retrieval and ranking
    - Automatic context compression
    """

    def __init__(self,
                 short_term_size: int = 10,
                 embedding_model: str = 'all-MiniLM-L6-v2',
                 collection_name: str = 'agent_memory'):

        print("🔧 Initializing Context Engineering Module...")

        # SHORT-TERM MEMORY: Rolling window for recent interactions
        self.short_term_memory = deque(maxlen=short_term_size)
        print(f"   ✓ Short-term memory: {short_term_size} message buffer")

        # LONG-TERM MEMORY: Vector database for semantic search
        self.chroma_client = chromadb.Client(Settings(
            anonymized_telemetry=False,
            allow_reset=True
        ))

        # Reset collection if exists
        try:
            self.chroma_client.delete_collection(collection_name)
        except:
            pass

        self.collection = self.chroma_client.create_collection(
            name=collection_name,
            metadata={"hnsw:space": "cosine"}
        )
        print(f"   ✓ Long-term memory: ChromaDB '{collection_name}'")

        # EMBEDDING MODEL: For semantic similarity
        self.embedding_model = SentenceTransformer(embedding_model)
        print(f"   ✓ Embedding model: {embedding_model}")

        # MEMORY COUNTER: For unique IDs
        self.memory_counter = 0

        print("Context Engineering Module ready!\n")

    def add_to_short_term(self, message: Dict[str, str]):
        """Add message to short-term memory (auto-expires old messages)"""
        message['timestamp'] = datetime.now().isoformat()
        self.short_term_memory.append(message)

    def add_to_long_term(self, text: str, metadata: Optional[Dict] = None):
        """Store important information in long-term vector memory"""
        if not text.strip():
            return

        # Generate embedding
        embedding = self.embedding_model.encode(text).tolist()

        # Store in vector DB
        self.memory_counter += 1
        self.collection.add(
            embeddings=[embedding],
            documents=[text],
            metadatas=[metadata or {}],
            ids=[f"mem_{self.memory_counter}"]
        )

    def retrieve_relevant_context(self, query: str, top_k: int = 5) -> List[str]:
        """Retrieve most relevant memories using semantic search"""
        if self.collection.count() == 0:
            return []

        # Generate query embedding
        query_embedding = self.embedding_model.encode(query).tolist()

        # Search vector database
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=min(top_k, self.collection.count())
        )

        return results['documents'][0] if results['documents'] else []

    def build_context_prompt(self,
                            user_query: str,
                            system_prompt: str = "",
                            include_short_term: bool = True,
                            include_long_term: bool = True) -> str:
        """Construct complete context for agent with all relevant information"""

        context_parts = []

        # Add system prompt
        if system_prompt:
            context_parts.append(f"SYSTEM: {system_prompt}")

        # Add long-term memory (most relevant historical context)
        if include_long_term:
            relevant_memories = self.retrieve_relevant_context(user_query, top_k=3)
            if relevant_memories:
                context_parts.append("\nRELEVANT KNOWLEDGE:")
                for i, mem in enumerate(relevant_memories, 1):
                    context_parts.append(f"{i}. {mem}")

        # Add short-term memory (recent conversation)
        if include_short_term and self.short_term_memory:
            context_parts.append("\nRECENT CONVERSATION:")
            for msg in list(self.short_term_memory)[-5:]:
                role = msg.get('role', 'unknown')
                content = msg.get('content', '')
                context_parts.append(f"{role.upper()}: {content}")

        # Add current query
        context_parts.append(f"\nCURRENT QUERY: {user_query}")

        return "\n".join(context_parts)

    def compress_context(self, text: str, max_length: int = 500) -> str:
        """Compress long context using extractive summarization"""
        if len(text) <= max_length:
            return text

        # Simple compression: take first and last portions
        half = max_length // 2
        return f"{text[:half]}... [TRUNCATED] ...{text[-half:]}"

    def get_memory_stats(self) -> Dict:
        """Get current memory statistics"""
        return {
            'short_term_messages': len(self.short_term_memory),
            'long_term_items': self.collection.count(),
            'total_memories': self.memory_counter
        }

# Initialize the context module
context_module = ContextEngineeringModule(short_term_size=10)
print(f"Memory Stats: {context_module.get_memory_stats()}")

🔧 Initializing Context Engineering Module...
   ✓ Short-term memory: 10 message buffer
   ✓ Long-term memory: ChromaDB 'agent_memory'
   ✓ Embedding model: all-MiniLM-L6-v2
✅ Context Engineering Module ready!

📊 Memory Stats: {'short_term_messages': 0, 'long_term_items': 0, 'total_memories': 0}


In [30]:
def create_enhanced_search_function(serper_tool, context_module):
    """
    Creates a simpler function-based search tool.
    This is the RECOMMENDED approach as it's more reliable with CrewAI.
    """
    from crewai.tools import tool

    @tool("enhanced_web_search")
    def enhanced_web_search(query: str) -> str:
        """
        Enhanced web search that integrates with the memory system.
        Uses SerperDevTool and stores results in long-term memory.

        Args:
            query: The search query string

        Returns:
            Formatted search results with memory integration
        """
        try:
            print(f"🔍 Executing search for: {query}")

            # ✅ CORRECT: Use search_query keyword argument
            results = serper_tool._run(search_query=query)

            # Handle dict response from Serper
            if isinstance(results, dict):
                formatted_results = []

                # Extract answer box
                if 'answerBox' in results:
                    answer = results['answerBox']
                    answer_text = answer.get('answer', answer.get('snippet', ''))
                    if answer_text:
                        formatted_results.append(f"Quick Answer: {answer_text}\n")

                # Extract knowledge graph
                if 'knowledgeGraph' in results:
                    kg = results['knowledgeGraph']
                    title = kg.get('title', '')
                    desc = kg.get('description', '')
                    if title or desc:
                        formatted_results.append(f"📚 Knowledge Graph: {title} - {desc}\n")

                # Extract organic results
                if 'organic' in results:
                    formatted_results.append("🔍 Top Search Results:\n")
                    for i, item in enumerate(results['organic'][:5], 1):
                        title = item.get('title', 'No title')
                        link = item.get('link', 'No link')
                        snippet = item.get('snippet', 'No description')
                        formatted_results.append(f"{i}. {title}")
                        formatted_results.append(f"   URL: {link}")
                        formatted_results.append(f"   {snippet}\n")

                results_str = "\n".join(formatted_results) if formatted_results else str(results)
            else:
                results_str = str(results)

            if not results_str or results_str.strip() == "":
                return f"No results found for query: {query}"

            # Store in long-term memory for RAG
            context_module.add_to_long_term(
                f"Search Query: {query}\nResults: {results_str}",
                metadata={
                    'source': 'web_search',
                    'query': query,
                    'timestamp': datetime.now().isoformat()
                }
            )

            return f"🔍 Search Results for: '{query}'\n\n{results_str}"

        except Exception as e:
            error_msg = f"Search error: {str(e)}"
            print(f"{error_msg}")
            print(f"   Error type: {type(e).__name__}")
            return f"Search failed: {error_msg}"

    return enhanced_web_search

In [31]:
print("Initializing CrewAI Tools...\n")

try:
    # 1. SerperDevTool - Web Search
    serper_tool = SerperDevTool()
    print("SerperDevTool initialized (Web Search)")

    # 2. ScrapeWebsiteTool - Website Scraping
    scrape_tool = ScrapeWebsiteTool()
    print("ScrapeWebsiteTool initialized (Website Scraping)")

    # 3. Enhanced Web Search Tool with robust input handling
    enhanced_web_search_tool = create_enhanced_search_function(serper_tool, context_module)
    print("Enhanced Web Search Tool initialized (Robust Input Handling)")

except Exception as e:
    print(f"Error initializing tools: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\nAll tools ready!\n")

🔧 Initializing CrewAI Tools...

✓ SerperDevTool initialized (Web Search)
✓ ScrapeWebsiteTool initialized (Website Scraping)
✓ Enhanced Web Search Tool initialized (Robust Input Handling)

✅ All tools ready!



In [33]:
# STEP 6: LLM CONFIGURATION


def create_llm(model_choice: str = "gemini"):
    """
    Create LLM instance using CrewAI's LLM class.

    Available models:
    - gemini: Google Gemini (FREE, recommended)
    - gemini-flash: Faster Gemini model
    - gemini-pro: More capable Gemini model
    """

    try:
        if model_choice == "gemini":
            print("🔧 Initializing Gemini 1.5 Flash (Fast & Free)...")
            llm = LLM(
                model="gemini-2.5-flash-lite",
                temperature=0.7,
                api_key=os.environ.get('GOOGLE_API_KEY'),
                max_retries=5,
                timeout=120,
                max_tokens=2048,
                request_timeout=60

            )
            print("LLM initialized: Gemini 1.5 Flash")

        elif model_choice == "gemini-pro":
            print("Initializing Gemini 1.5 Pro (More Capable)...")
            llm = LLM(
                model="gemini/gemini-1.5-pro",
                temperature=0.7,
                api_key=os.environ.get('GOOGLE_API_KEY')
            )
            print("LLM initialized: Gemini 1.5 Pro")

        else:
            print("Initializing default Gemini model...")
            llm = LLM(
                model="gemini/gemini-pro",
                temperature=0.7,
                api_key=os.environ.get('GOOGLE_API_KEY')
            )
            print("LLM initialized: Gemini Pro")

        return llm

    except Exception as e:
        print(f"Error initializing LLM: {e}")
        print("   Please check your GEMINI_API_KEY")
        raise

# Initialize LLM with Gemini
# Options: "gemini" (flash), "gemini-pro", or default
llm = create_llm("gemini")  # Using Gemini 1.5 Flash (fast & free)

🔧 Initializing Gemini 1.5 Flash (Fast & Free)...
✅ LLM initialized: Gemini 1.5 Flash


In [34]:
# MULTI-AGENT ARCHITECTURE


print("\nBuilding Multi-Agent System...\n")

# AGENT 1: RESEARCH AGENT
research_agent = Agent(
    role='Research Specialist',
    goal='Search and retrieve the most relevant and accurate information from the web',
    backstory="""You are an expert research analyst with exceptional skills in:
    - Formulating effective search queries
    - Evaluating source credibility and relevance
    - Extracting key information from multiple sources
    - Identifying patterns and connections across documents
    - Maintaining objectivity and avoiding bias

    You always prioritize accuracy and cite your sources clearly.""",
    tools=[enhanced_web_search_tool],
    llm=llm,
    verbose=True,
    allow_delegation=False,
    max_iter=1,  # REDUCED from 3 to 1 to save API calls
    max_rpm=10
)
print("Research Agent created (powered by Gemini)")

# AGENT 2: ANALYSIS AGENT
analysis_agent = Agent(
    role='Critical Analysis Expert',
    goal='Synthesize information, detect contradictions, and provide deep analytical insights',
    backstory="""You are a brilliant analyst specializing in:
    - Critical thinking and logical reasoning
    - Identifying contradictions and inconsistencies
    - Synthesizing complex information into clear insights
    - Cross-referencing multiple sources for accuracy
    - Detecting potential biases or hallucinations

    You never make unfounded claims and always acknowledge uncertainty when appropriate.""",
    tools=[],
    llm=llm,
    verbose=True,
    allow_delegation=False,
    max_iter=1,  # REDUCED from 3 to 1 to save API calls
    max_rpm=10
)
print("Analysis Agent created (powered by Gemini)")

# AGENT 3: WRITING AGENT
writing_agent = Agent(
    role='Professional Content Writer',
    goal='Transform analyzed information into clear, accurate, and engaging final output',
    backstory="""You are an expert technical writer with skills in:
    - Clear and concise communication
    - Maintaining factual accuracy while being engaging
    - Structuring information logically
    - Adapting tone and style to the audience
    - Ensuring context integrity and coherence

    You transform complex analyses into accessible, well-structured responses.""",
    tools=[],
    llm=llm,
    verbose=True,
    allow_delegation=False,
    max_iter=1,  # REDUCED from 3 to 1 to save API calls
    max_rpm=10
)
print("Writing Agent created (powered by Gemini)\n")

print("All agents initialized successfully!")



🗂️ Building Multi-Agent System...

✓ Research Agent created (powered by Gemini)
✓ Analysis Agent created (powered by Gemini)
✓ Writing Agent created (powered by Gemini)

✅ All agents initialized successfully!


In [38]:
def create_agent_tasks(user_query: str, context_summary: str, research_agent, analysis_agent, writing_agent):
    """
    Creates properly configured tasks that prevent the analysis agent from rejecting valid research.
    """

    # TASK 1: RESEARCH TASK
    research_task = Task(
        description=f"""CRITICAL: You MUST use the enhanced_web_search tool to find REAL information.

        Query: {user_query}

        Context from memory: {context_summary}

        MANDATORY STEPS:
        1. Call enhanced_web_search tool with the query
        2. Wait for REAL results from the tool
        3. Extract and summarize the actual search results
        4. DO NOT fabricate or simulate any data
        5. If tool fails, report the error explicitly

        FORBIDDEN:
        - DO NOT use internal knowledge without tool verification
        - DO NOT make up search results
        - DO NOT proceed without calling the tool
        """,
        agent=research_agent,
        expected_output="Real search results from the enhanced_web_search tool with source citations, URLs, and content snippets"
    )

    # TASK 2: ANALYSIS TASK
    analysis_task = Task(
        description=f"""You are analyzing web search results provided by the Research Specialist.

        IMPORTANT CLARIFICATIONS:
        - "Verified research data" = Web search summaries, URLs, and snippets from the previous agent
        - You ARE NOT expecting: academic papers, raw datasets, or scientific studies
        - You ARE expecting: Search engine results with titles, URLs, and text snippets
        - The previous agent's output IS the research data - do not claim it's missing

        YOUR JOB:
        1. READ the search results provided by the Research Specialist above
        2. EXTRACT key information from the titles, URLs, and snippets
        3. SYNTHESIZE the main points into a coherent analysis
        4. IDENTIFY patterns or themes across multiple search results
        5. NOTE any contradictions between different sources
        6. FLAG genuine limitations (e.g., "sources only cover X, not Y")

        FORBIDDEN BEHAVIORS:
        - DO NOT claim "no data was provided" when search results are present
        - DO NOT reject URLs/snippets as "not real research data"
        - DO NOT demand academic papers or datasets
        - DO NOT hallucinate that information is missing when it's clearly there
        - DO NOT add information from your own knowledge - use only what's provided

        ACCEPTABLE LIMITATIONS TO NOTE:
        - "Search results are limited to top 5 matches"
        - "No specific statistics were found, only general descriptions"
        - "Sources focus on X aspect but don't cover Y aspect"

        Query being analyzed: {user_query}

        Now analyze the search results provided by the Research Specialist above.
        """,
        agent=analysis_agent,
        expected_output="""A synthesis of the web search results that:
        - Summarizes key information from the titles, URLs, and snippets
        - Identifies main themes or patterns
        - Notes any contradictions between sources
        - Acknowledges actual limitations (not fabricated ones)
        - DOES NOT claim data is missing when search results were provided""",
        context=[research_task]
    )

    # TASK 3: WRITING TASK
    writing_task = Task(
        description=f"""Create a comprehensive answer based on the research and analysis provided above.

        Query: {user_query}

        YOUR SOURCES:
        1. Research Specialist's web search results (titles, URLs, snippets)
        2. Analysis Expert's synthesis of those results

        REQUIREMENTS:
        1. Use ONLY information from the research and analysis above
        2. Structure the answer logically with clear sections
        3. Include specific details from the search results
        4. Cite sources by mentioning URLs or source names when relevant
        5. If the analysis noted limitations, acknowledge them briefly
        6. DO NOT add external information from your own knowledge
        7. DO NOT claim information is missing if it was provided in earlier steps

        FORMAT:
        - Start with a direct answer to the query
        - Provide supporting details from the search results
        - Include 2-3 relevant source URLs if available
        - End with a brief summary or key takeaway

        Tone: Clear, informative, and factual
        """,
        agent=writing_agent,
        expected_output="""A well-structured answer that:
        - Directly addresses the user's query
        - Uses specific information from the web search results
        - Includes source citations (URLs)
        - Is accurate and based solely on provided research
        - Acknowledges real limitations if they exist""",
        context=[research_task, analysis_task]
    )

    return research_task, analysis_task, writing_task


In [39]:
def process_query(user_query: str, use_memory: bool = True) -> str:
    """
    Complete pipeline with fixed task definitions.
    """

    print("\n" + "="*70)
    print("STARTING MULTI-AGENT PIPELINE")
    print("="*70 + "\n")

    try:
        # STEP 1: CONTEXT ENGINEERING
        print("Step 1: Context Retrieval...")

        if use_memory:
            relevant_context = context_module.retrieve_relevant_context(user_query, top_k=3)
            context_summary = "\n".join(relevant_context) if relevant_context else "No relevant historical context."
            print(f"   Retrieved {len(relevant_context)} relevant memories")
        else:
            context_summary = "No historical context used."
            print("   Skipping memory retrieval")

        context_module.add_to_short_term({
            'role': 'user',
            'content': user_query
        })

        # STEP 2: CREATE AGENT TASKS WITH FIXED DEFINITIONS
        print("\nStep 2: Creating Agent Tasks with Fixed Logic...")

        research_task, analysis_task, writing_task = create_agent_tasks(
            user_query=user_query,
            context_summary=context_summary,
            research_agent=research_agent,
            analysis_agent=analysis_agent,
            writing_agent=writing_agent
        )

        # STEP 3: CREATE AND RUN CREW
        print("\n Step 3: Assembling Crew...")

        crew = Crew(
            agents=[research_agent, analysis_agent, writing_agent],
            tasks=[research_task, analysis_task, writing_task],
            process=Process.sequential,
            verbose=True,
            max_rpm=10,
            share_crew=False
        )

        print("\nStep 4: Executing Multi-Agent Workflow...\n")

        # Execute
        result = crew.kickoff()
        final_answer = str(result)

        # UPDATE MEMORY
        print("\nStep 5: Updating Memory...")

        context_module.add_to_short_term({
            'role': 'assistant',
            'content': final_answer
        })

        context_module.add_to_long_term(
            f"Query: {user_query}\nAnswer: {final_answer}",
            metadata={'type': 'qa_pair', 'timestamp': datetime.now().isoformat()}
        )

        stats = context_module.get_memory_stats()
        print(f"   Short-term: {stats['short_term_messages']} messages")
        print(f"   Long-term: {stats['long_term_items']} items stored")

        print("\n" + "="*70)
        print("PIPELINE COMPLETE")
        print("="*70 + "\n")

        return final_answer

    except Exception as e:
        error_msg = f"Error in pipeline execution: {str(e)}"
        print(f"\n{error_msg}")
        print(f"Error type: {type(e).__name__}")

        import traceback
        traceback.print_exc()

        return f"Pipeline failed: {error_msg}"




In [40]:
def safe_test_query(query: str):
    """Wrapper to safely test queries with error handling."""
    try:
        print("\n" + "#"*70)
        print(f"# TEST QUERY: {query}")
        print("#"*70)

        answer = process_query(query)

        print("\n" + "="*70)
        print("FINAL ANSWER:")
        print("="*70)
        print(answer)

        # Wait between queries to respect rate limits
        print("\n⏱️  Waiting 10 seconds before next query...")
        time.sleep(10)

        return answer

    except KeyboardInterrupt:
        print("\nQuery interrupted by user")
        return None
    except Exception as e:
        print(f"\nTest query failed: {e}")
        return None

In [ ]:
answer = safe_test_query("What is CrewAI?")

In [ ]:
# Example 1: Technical Explanation
print("\n" + "#"*70)
print("# EXAMPLE 1: Retrieval-Augmented Generation (RAG)")
print("#"*70)

answer1 = process_query(
    "Explain retrieval-augmented generation (RAG) and how it improves LLM responses"
)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(answer1)

In [ ]:


# Example 2: Current Research
print("\n" + "#"*70)
print("# EXAMPLE 2: Multi-Agent Systems Research")
print("#"*70)

answer2 = process_query(
    "What are the latest developments in multi-agent AI systems?"
)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(answer2)

# Example 3: Concept Definition
print("\n" + "#"*70)
print("# EXAMPLE 3: Context Engineering")
print("#"*70)

answer3 = process_query(
    "What is context engineering in AI systems and why is it important?"
)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(answer3)

# Example 4: Framework Comparison
print("\n" + "#"*70)
print("# EXAMPLE 4: CrewAI vs AutoGen")
print("#"*70)

answer4 = process_query(
    "Compare CrewAI and AutoGen frameworks for building multi-agent systems"
)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(answer4)

# Example 5: Technical Architecture
print("\n" + "#"*70)
print("# EXAMPLE 5: LLM Memory Architectures")
print("#"*70)

answer5 = process_query(
    "Describe different memory architectures used in Large Language Models"
)

print("\n" + "="*70)
print("FINAL ANSWER:")
print("="*70)
print(answer5)


In [42]:


# ================================================================================
# STEP 10: MEMORY VISUALIZATION AND STATS
# ================================================================================

import pandas as pd

# Get memory statistics
stats = context_module.get_memory_stats()

print("\n" + "="*70)
print("MEMORY SYSTEM STATISTICS")
print("="*70 + "\n")

print(f"Short-Term Memory:")
print(f"  └─ Current Messages: {stats['short_term_messages']}/{context_module.short_term_memory.maxlen}")
print(f"\nLong-Term Memory (Vector DB):")
print(f"  ├─ Total Items Stored: {stats['long_term_items']}")
print(f"  └─ Total Memories Created: {stats['total_memories']}")

# Display recent short-term memory
if context_module.short_term_memory:
    print("\n" + "-"*70)
    print("Recent Conversation History:")
    print("-"*70 + "\n")

    for i, msg in enumerate(list(context_module.short_term_memory)[-5:], 1):
        role = msg.get('role', 'unknown').upper()
        content = msg.get('content', '')[:100] + "..." if len(msg.get('content', '')) > 100 else msg.get('content', '')
        timestamp = msg.get('timestamp', 'N/A')
        print(f"{i}. [{role}] {timestamp}")
        print(f"   {content}\n")

print("="*70)




MEMORY SYSTEM STATISTICS

Short-Term Memory:
  └─ Current Messages: 8/10

Long-Term Memory (Vector DB):
  ├─ Total Items Stored: 6
  └─ Total Memories Created: 6

----------------------------------------------------------------------
Recent Conversation History:
----------------------------------------------------------------------

1. [ASSISTANT] 2025-11-30T04:20:18.364643
   Based on the provided context, no verified research data or search results regarding "CrewAI" were s...

2. [USER] 2025-11-30T04:23:28.175175
   What is CrewAI?

3. [ASSISTANT] 2025-11-30T04:23:34.851052
   Based on the provided context, CrewAI is described as a leading multi-agent platform and a standalon...

4. [USER] 2025-11-30T04:42:31.053389
   What is CrewAI?

5. [ASSISTANT] 2025-11-30T04:42:44.870108
   CrewAI is an open-source, Python-based framework designed for orchestrating autonomous AI agents. It...

